```{contents}
```
## Planner–Executor Pattern 

The **Planner–Executor** is one of the most powerful architectural patterns in LangGraph for building **intelligent, multi-step, autonomous systems**.
It separates **high-level reasoning** from **low-level execution**, yielding better reliability, scalability, and interpretability.

---

### **1. Core Intuition**

Instead of letting an LLM directly perform a task end-to-end, we divide cognition into two roles:

| Role           | Responsibility                          |
| -------------- | --------------------------------------- |
| **Planner**    | Decomposes goal into structured steps   |
| **Executor**   | Carries out each step deterministically |
| **Controller** | Coordinates planning and execution      |

This mimics how humans solve complex problems:
**think first, act second**.

---

### **2. Conceptual Workflow**

```
User Goal
   ↓
Planner ──► Structured Plan
               ↓
          Executor (step 1)
               ↓
          Executor (step 2)
               ↓
            ...
               ↓
         Validation / Feedback
               ↓
       Replan if necessary
               ↺
```

This loop continues until the goal is satisfied.

---

### **3. Why LangGraph Fits This Pattern Perfectly**

LangGraph natively provides:

* **Stateful memory** for plan storage
* **Cyclic graphs** for iterative planning
* **Conditional routing** for replanning
* **Tool execution nodes**
* **Human-in-the-loop hooks**

---

### **4. State Design**

```python
class State(TypedDict):
    goal: str
    plan: list[str]
    step_index: int
    result: dict
    done: bool
```

---

### **5. Planner Node**

```python
def planner(state):
    prompt = f"Break this goal into steps: {state['goal']}"
    plan = llm.invoke(prompt)
    return {"plan": parse_steps(plan), "step_index": 0}
```

The planner runs **only when needed**.

---

### **6. Executor Node**

```python
def executor(state):
    step = state["plan"][state["step_index"]]
    output = execute_step(step)
    return {"result": output, "step_index": state["step_index"] + 1}
```

---

### **7. Controller & Looping Logic**

```python
def should_continue(state):
    if state["step_index"] >= len(state["plan"]):
        return END
    return "execute"
```

```python
builder.add_conditional_edges("execute", should_continue, {
    "execute": "execute",
    END: END
})
```

This creates a **controlled execution loop**.

---

### **8. Full LangGraph Assembly**

```python
builder = StateGraph(State)

builder.add_node("plan", planner)
builder.add_node("execute", executor)

builder.set_entry_point("plan")
builder.add_edge("plan", "execute")

builder.add_conditional_edges("execute", should_continue, {
    "execute": "execute",
    END: END
})

graph = builder.compile()
```

---

### **9. Variants of Planner–Executor**

| Variant         | Description                     |
| --------------- | ------------------------------- |
| Single-pass     | Plan once, execute once         |
| Iterative       | Replan after each execution     |
| Hierarchical    | Planner generates sub-plans     |
| Self-reflective | Executor feedback improves plan |
| Multi-agent     | Multiple executors per planner  |

---

### **10. Advantages Over Linear Chains**

| Linear Chain  | Planner–Executor |
| ------------- | ---------------- |
| Rigid         | Adaptive         |
| Opaque        | Interpretable    |
| Error-prone   | Self-correcting  |
| Single-shot   | Multi-step       |
| Hard to debug | Fully traceable  |

---

### **11. Production Use Cases**

* Autonomous research agents
* Code generation & debugging systems
* Business workflow automation
* AI copilots
* Data analysis pipelines
* Operations automation

---

### **12. Safety & Reliability Controls**

* Maximum step limits
* Validation after each step
* Human approval gates
* Checkpointing
* Rollback & retry

---

### **13. Mental Model**

LangGraph implements the Planner–Executor as a **feedback-controlled state machine**:

> **Plan → Execute → Observe → Adjust → Repeat**

This is the foundation of **modern autonomous AI systems**.

---

### Demonstration

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# -----------------------------
# 1. Define Shared State
# -----------------------------
class State(TypedDict):
    goal: str
    plan: List[str]
    step_index: int
    results: List[str]

# -----------------------------
# 2. Planner Node
# -----------------------------
def planner(state: State):
    goal = state["goal"]
    # (Mock LLM output for demo)
    plan = [
        f"Research topic: {goal}",
        "Summarize findings",
        "Write final answer"
    ]
    return {"plan": plan, "step_index": 0, "results": []}

# -----------------------------
# 3. Executor Node
# -----------------------------
def executor(state: State):
    step = state["plan"][state["step_index"]]
    result = f"Completed -> {step}"
    
    new_results = state["results"] + [result]
    return {
        "results": new_results,
        "step_index": state["step_index"] + 1
    }

# -----------------------------
# 4. Loop Controller
# -----------------------------
def controller(state: State):
    if state["step_index"] >= len(state["plan"]):
        return END
    return "execute"

# -----------------------------
# 5. Build LangGraph
# -----------------------------
builder = StateGraph(State)

builder.add_node("plan", planner)
builder.add_node("execute", executor)

builder.set_entry_point("plan")
builder.add_edge("plan", "execute")

builder.add_conditional_edges("execute", controller, {
    "execute": "execute",
    END: END
})

graph = builder.compile()

# -----------------------------
# 6. Run Graph
# -----------------------------
final_state = graph.invoke({"goal": "Explain black holes"})

print("\nExecution Results:\n")
for r in final_state["results"]:
    print(r)



Execution Results:

Completed -> Research topic: Explain black holes
Completed -> Summarize findings
Completed -> Write final answer
